# Import libraries

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

# Load data

In [ ]:
df_demo = pd.read_csv('Data/df_demography.csv', sep=',', dtype={'cpr': str})
df_pers = pd.read_csv('Data/df_personal_characteristic.csv', sep=',', dtype={'cpr': str})

In [ ]:
df_pers.to_pickle('Data/df_personal_characteristic.pkl')

In [ ]:
df_pers = pd.read_pickle('Data/df_personal_characteristic.pkl')

# Data preprocessing / Feature engineering

#### Merge data from different sources

In [ ]:
df_demo.head(5)

In [ ]:
df_pers.head(5)

In [ ]:
df = pd.merge(df_demo, df_pers, on='cpr')

In [ ]:
df.head(5)

In [ ]:
len(df)

#### Generate new variables

In [ ]:
df['gender'] = df['cpr'].str[9].apply(int)
df['gender'] = np.where(df['gender'] % 2 == 0, 'woman', 'man')

In [ ]:
df.head(5)

In [ ]:
df['year'] = df['cpr'].str[4:6]
df['year'] = '19' + df['year']

In [ ]:
df['month'] = df['cpr'].str[2:4]

In [ ]:
df['day'] = df['cpr'].str[0:2]

In [ ]:
df['birthday'] = pd.to_datetime(df['day'] + df['month'] + df['year'], format='%d%m%Y')

In [ ]:
df.head(5)

In [ ]:
from datetime import datetime

df['age'] = datetime.now() - df['birthday']
df['age'] = df['age'].astype('timedelta64[Y]')

In [ ]:
df.head(5)

In [ ]:
df = df.iloc[:1500]

# Split into train and test set

In [ ]:
X = df[['weight', 'height']].values
y = df['gender'].values

In [ ]:
X

In [ ]:
y

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

# Build and train model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(model.n_neighbors)
print(model.classes_)

# Make predictions

In [ ]:
prediction = model.predict([[81, 1.81]]) # [81, 1.81]
print(prediction)

In [ ]:
prediction_probability = model.predict_proba([[80, 1.7]])
print(prediction_probability)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

# Performance metrics

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report

In [ ]:
print(y_test[:10])
print(y_pred[:10])

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.figure(figsize=(8, 8))
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list('some_name', ['#ffffff', '#35af5d'], N=256, gamma=1.0)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    #thresh = max(cm.max() / 2., 0)
    thresh = 1
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if cm[i, j] > thresh:
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     verticalalignment="center",
                     color="black",
                     fontsize=12)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['No', 'Yes'])

In [ ]:
accuracy_score(y_test, y_pred)

# sum(y_test == y_pred) / len(y_test)

In [ ]:
precision_score(y_test, y_pred, pos_label='woman')

# tp / (tp + fp)

In [ ]:
recall_score(y_test, y_pred, pos_label='woman')

# tp / (tp + fn)

In [ ]:
print(classification_report(y_test, y_pred))

# Visualize

In [ ]:
X_test[:10]

In [ ]:
y_test[:10]

In [ ]:
df[['weight', 'height']].head(5)

In [ ]:
weights = X_test[:,0]

heights = X_test[:,1]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
plt.scatter(weights, heights, c='k', marker='.')
plt.axis([40, 110, 1.4, 2.2])
plt.xlabel('weight')
plt.ylabel('height')
plt.title('How is data distributed?')
plt.show()

In [ ]:
# y_test == 'woman'

In [ ]:
color_true = np.where(y_test == 'woman', 'b', 'r')
color_pred = np.where(y_pred == 'woman', 'c', 'm')

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(weights, heights, c=color_true, marker='.')
plt.axis([40, 110, 1.4, 2.2])
plt.xlabel('weight')
plt.ylabel('height')
plt.title('How is data distributed?')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(weights, heights, c=color_pred, marker='.')
plt.axis([40, 110, 1.4, 2.2])
plt.xlabel('weight')
plt.ylabel('height')
plt.title('How is data distributed?')
plt.show()

# Øvelser - Fine tune model

- Prøv at træne modellen med forskellige antal naboer ($n$-værdier) og se hvordan det påvirker performance metrics.
- Prøv at tilføje en ny kolonne til træningsdata. Dvs. tilføj også ```salary``` til $X$ og træn modellen igen. Se hvordan det påvirker performance metrics.
- Importer en anden klassifikationsmodel og lav prædiktioner med denne på samme måde som overstående. Brug logistisk regression fra scikit learn.
  
  ```from sklearn.linear_model import LogisticRegression```

# Exercise - Fine tune model

- Try training the model with different numbers of neighbors ($n$) and see how it affects the performance metrics.
- Try adding a new column to the training data. I.e. add the ```salary``` column and retrain the model. See how it affects the performance metrics.
- Import another classification model and make prediction in the same way as above. Use logistic regression from scikit learn.
  
  ```from sklearn.linear_model import LogisticRegression```